In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from skimage.io import imsave

from biobeam import SimLSM_Cylindrical
from biobeam.data import tiling
import gputools

from sim_util import sim_lightsheet_img

%matplotlib inline

# set and check which gpu is used
gputools.init_device(id_platform=0, id_device=1)
gputools.get_device()

In [ ]:
def get_centered_padded(a, loc, out_shape, pad_val=None):
    """
    
    Parameters
    ----------
    a: array
        array to center/cut
    loc: 1d-array-like
        center of cut
    out_shape:
        shape of produced output
    """
    
    res = np.full(out_shape, 0 if pad_val is None else pad_val, dtype=a.dtype)
    
    # get window in a centered on loc 
    max_ = np.floor(np.array(loc) + (np.array(out_shape))/2) 
    min_ = np.ceil(np.array(loc) - (np.array(out_shape))/2)

    max_cut = np.min(np.stack([np.array(a.shape), max_]), axis=0).astype(np.int)
    min_cut = np.max(np.stack([np.zeros_like(min_), min_]), axis=0).astype(np.int)

    window = a[tuple([slice(mi, ma) for mi, ma in zip(min_cut, max_cut)])]
    
    print(window.shape, min_cut, max_cut)

    # where to copy to in output
    center_out = (np.array(out_shape)-1) / 2
    max_res = (np.floor(center_out + (max_ - min_)/2) - (max_ - max_cut )).astype(np.int)
    min_res = (np.ceil(center_out - (max_ - min_)/2) + (min_cut - min_ )).astype(np.int)

    print(center_out, min_res, max_res)
    copy_to = res[tuple([slice(mi, ma) for mi, ma in zip(min_res, max_res)])]
    copy_to[:] = window

    return res
    
    
def get_centered_padded2(a, off, out_shape, pad_val=None):
    """
    simpler version of centered/padded image
    
    """
    
    res = np.full(out_shape, 0 if pad_val is None else pad_val, dtype=a.dtype)
    minmax = np.min(np.stack([np.array(a.shape), np.array(res.shape)]), axis=0)
    siz_dif = np.array(res.shape) - np.array(a.shape)
    
    slices = tuple([slice(0, ma) for ma in minmax])
    res[slices] = a[slices]
    
    for i,d in enumerate(siz_dif):
        res = np.roll(res, int(d/2 + off[i]), i)

    return res

    

loc = [0, 10]
out_shape = [10, 30]
a = np.ones((10, 10))

res = get_centered_padded2(a, loc, out_shape)

plt.imshow(res)
#res.shape


#np.argmax(res[-1])

In [ ]:
# helper for saving tiff stacks

from PIL import Image

def save_tiff_stack(arr, fname, axis=0):
    """
    save array as tiff stack

    Parameters
    ----------
    arr: array
        3d-array to save
    fname: string
        path of output file
    axis: int, optional
        along which axis to create slices
    """

    def make_index_(i):
        return tuple([i if j == axis else slice(d) for j, d in enumerate(arr.shape)])

    imlist = [Image.fromarray(arr[make_index_(i)]) for i in range(arr.shape[axis])]
    imlist[0].save(fname, save_all=True,
                   append_images=imlist[1:])

In [ ]:
N = 256
phys_dim = 100
r_sphere = 40
grid_spacing = 10
ri_diff = 0.1
z_subsampling = 4

# half-sphere with higher ri
x = np.linspace(-phys_dim/2, phys_dim/2, N)
Xs = np.meshgrid(*([x]*3))
R = np.sqrt(np.sum([_X**2 for _X in Xs], axis = 0))
mask = (R<r_sphere) * (Xs[1] > 0)
dn = ri_diff * mask

# grid signal
x = np.arange(0, N)
Xs = np.meshgrid(*([x]*3))
grid = np.prod([_X%grid_spacing==0 for _X in Xs], axis = 0).astype(np.float32)

In [ ]:
Nx = 512
grid_left = get_centered_padded2(grid, [0,0,-127], [N,N,Nx])
grid_right = get_centered_padded2(grid, [0,0,128], [N,N,Nx])
dn_left = get_centered_padded2(dn, [0,0,-127], [N,N,Nx])
dn_right = get_centered_padded2(dn, [0,0,128], [N,N,Nx])


# sim with left and right illum
image_left, _ = sim_lightsheet_img(img=grid_left, desc=None, dn=dn_left, bprop_nvolumes=1, conv_subblocks=(1,1), padding=32,
                                   right_illum=False, na_illum=0.1,
                                   na_detect=0.5, physical_dims=(phys_dim*2, phys_dim, phys_dim),
                                   ls_pos=phys_dim, lam=500,
                                   z_plane_subset=np.arange(0, N, z_subsampling))

image_right, _ = sim_lightsheet_img(img=grid_right, desc=None, dn=dn_right, bprop_nvolumes=1, conv_subblocks=(1,1), padding=32,
                                   right_illum=False, na_illum=0.1,
                                   na_detect=0.5, physical_dims=(phys_dim*2, phys_dim, phys_dim),
                                   ls_pos=phys_dim, lam=500,
                                   z_plane_subset=np.arange(0, N, z_subsampling))


save_tiff_stack(image_left, '/scratch/hoerl/sphere_test/test_moved_left.tif')
save_tiff_stack(image_right, '/scratch/hoerl/sphere_test/test_moved_right.tif')

In [ ]:
# sim with left and right illum
image_left, _ = sim_lightsheet_img(img=grid, desc=None, dn=dn, bprop_nvolumes=1, conv_subblocks=(1,1), padding=32,
                                   right_illum=False, na_illum=0.1,
                                   na_detect=0.5, physical_dims=(phys_dim,)*3,
                                   ls_pos=phys_dim/2, lam=500,
                                   z_plane_subset=np.arange(0, N, z_subsampling))

image_right, _ = sim_lightsheet_img(img=grid, desc=None, dn=dn, bprop_nvolumes=1, conv_subblocks=(1,1), padding=32,
                                   right_illum=True, na_illum=0.1,
                                   na_detect=0.5, physical_dims=(phys_dim,)*3,
                                   ls_pos=phys_dim/2, lam=500,
                                   z_plane_subset=np.arange(0, N, z_subsampling))


save_tiff_stack(image_left, '/scratch/hoerl/sphere_test/test_illum_left.tif')
save_tiff_stack(image_right, '/scratch/hoerl/sphere_test/test_illum_right.tif')

In [ ]:
# sim with lightsheet focus at 1/4 x and 3/4 x
image_ls_left, _ = sim_lightsheet_img(img=grid, desc=None, dn=dn, bprop_nvolumes=1, conv_subblocks=(1,1), padding=32,
                                   right_illum=False, na_illum=0.1,
                                   na_detect=0.5, physical_dims=(phys_dim,)*3,
                                   ls_pos=phys_dim/4, lam=500,
                                   z_plane_subset=np.arange(0, N, z_subsampling))

image_ls_right, _ = sim_lightsheet_img(img=grid, desc=None, dn=dn, bprop_nvolumes=1, conv_subblocks=(1,1), padding=32,
                                   right_illum=True, na_illum=0.1,
                                   na_detect=0.5, physical_dims=(phys_dim,)*3,
                                   ls_pos=phys_dim*3/4, lam=500,
                                   z_plane_subset=np.arange(0, N, z_subsampling))

save_tiff_stack(image_ls_left, '/scratch/hoerl/sphere_test/test_lsfocus_left.tif')
save_tiff_stack(image_ls_right, '/scratch/hoerl/sphere_test/test_lsfocus_right.tif')

In [ ]:
# sim at 400 & 600nm
image400nm, _ = sim_lightsheet_img(img=grid, desc=None, dn=dn, bprop_nvolumes=1, conv_subblocks=(1,1), padding=32,
                                   right_illum=False, na_illum=0.1,
                                   na_detect=0.5, physical_dims=(phys_dim,)*3,
                                   ls_pos=phys_dim/2, lam=400,
                                   z_plane_subset=np.arange(0, N, z_subsampling))

image600nm, _ = sim_lightsheet_img(img=grid, desc=None, dn=dn, bprop_nvolumes=1, conv_subblocks=(1,1), padding=32,
                                   right_illum=True, na_illum=0.1,
                                   na_detect=0.5, physical_dims=(phys_dim,)*3,
                                   ls_pos=phys_dim/2, lam=600,
                                   z_plane_subset=np.arange(0, N, z_subsampling))

save_tiff_stack(image400nm, '/scratch/hoerl/sphere_test/test_400nm.tif')
save_tiff_stack(image600nm, '/scratch/hoerl/sphere_test/test_600nm.tif')

In [ ]:
# plot left/right illum
fig, axs = plt.subplots(ncols=3, figsize=[20,20])
image_left_ = image_left[N//z_subsampling//2]
image_right_ = image_left[N//z_subsampling//2]
axs[0].imshow(1.5 * np.stack([image_left_/np.max(image_left_), image_right_/np.max(image_right_), np.zeros_like(image_left_)], 2))
axs[0].set_title('red: left, green: right')
axs[1].imshow(image_left_**(1/1), cmap='gray')
axs[1].set_title('left')
axs[2].imshow(image_right_**(1/1), cmap='gray')
axs[2].set_title('right')
fig.suptitle('left/right illumination', fontsize=20, y=0.68)
plt.savefig('/scratch/hoerl/sphere_test/left_right.png', dpi=400)


# plot colors
image400nm_ = image400nm[N//z_subsampling//2]
image600nm_ = image600nm[N//z_subsampling//2]
fig, axs = plt.subplots(ncols=3, figsize=[20,20])
axs[0].imshow(1.5 * np.stack([image400nm_/np.max(image400nm_), image600nm_/np.max(image600nm_), np.zeros_like(image400nm_)], 2))
axs[0].set_title('red: 400nm, green: 600nm')
axs[1].imshow(image400nm_**(1/1), cmap='gray')
axs[1].set_title('400 nm')
axs[2].imshow(image600nm_**(1/1), cmap='gray')
axs[2].set_title('600 nm')
fig.suptitle('wavelengths', fontsize=20, y=0.68)
plt.savefig('/scratch/hoerl/sphere_test/lambda.png', dpi=400)


# plot colors
image_ls_left_ = image_ls_left[N//z_subsampling//2]
image_ls_right_ = image_ls_right[N//z_subsampling//2]
fig, axs = plt.subplots(ncols=3, figsize=[20,20])
axs[0].imshow(1.5 * np.stack([image_ls_left_/np.max(image_ls_left_), image_ls_right_/np.max(image_ls_right_), np.zeros_like(image_ls_left_)], 2))
axs[0].set_title('red: 1/4 x, green: 3/4 x')
axs[1].imshow(image_ls_left_**(1/1), cmap='gray')
axs[1].set_title('1/4 x')
axs[2].imshow(image_ls_right_**(1/1), cmap='gray')
axs[2].set_title('3/4 x')
fig.suptitle('left right lightsheet focus', fontsize=20, y=0.68)
plt.savefig('/scratch/hoerl/sphere_test/ls_pos.png', dpi=400)


In [ ]:
# old code
# only single plane sim here

In [ ]:
# sim from left
m = SimLSM_Cylindrical(dn=dn, signal=grid, NA_illum=0.1, NA_detect=0.5, size=(phys_dim,)*3, n0=1.33)
image_left = m.simulate_image_z(cz=0, psf_grid_dim=(16,16),conv_sub_blocks=(2,2))[16]


image_left, _ = sim_lightsheet_img(img=grid, desc=None, dn=dn,
                                   right_illum=False, na_illum==0.1,
                                   na_detect=0.5, physical_dims=(phys_dim,)*3,
                                   ls_pos=phys_dim/2, lam=500)

# sim from right
m = SimLSM_Cylindrical(dn=np.flip(dn,2), signal=np.flip(grid,2), NA_illum=0.1, NA_detect=0.5, size=(phys_dim,)*3, n0=1.33)
image_right = m.simulate_image_z(cz=0, psf_grid_dim=(16,16),conv_sub_blocks=(2,2))[16]
# flip back
image_right = np.flip(image_right, 1)

imsave('/Users/david/Desktop/test_illum_left.tif', image_left)
imsave('/Users/david/Desktop/test_illum_right.tif', image_right)

In [ ]:
# sim left ls pos
m = SimLSM_Cylindrical(dn=dn, signal=grid, NA_illum=0.1, NA_detect=0.4, size=(phys_dim,)*3, n0=1.33,
                       zfoc_illum=phys_dim/4)
image_ls_left = m.simulate_image_z(cz=0, psf_grid_dim=(16,16),conv_sub_blocks=(2,2))[16]

# sim right ls pos
m = SimLSM_Cylindrical(dn=dn, signal=grid, NA_illum=0.1, NA_detect=0.4, size=(phys_dim,)*3, n0=1.33,
                       zfoc_illum=phys_dim*3/4)
image_ls_right = m.simulate_image_z(cz=0, psf_grid_dim=(16,16),conv_sub_blocks=(2,2))[16]

imsave('/Users/david/Desktop/test_lsfocus_left.tif', image_ls_left)
imsave('/Users/david/Desktop/test_lsfocus_right.tif', image_ls_right)

In [ ]:
# sim middle at 400nm
m = SimLSM_Cylindrical(dn=dn, signal=grid, NA_illum=0.1, NA_detect=0.4, size=(phys_dim,)*3, n0=1.33,
                       lam_detect=0.4, lam_illum=0.4)
image400nm = m.simulate_image_z(cz=0, psf_grid_dim=(16,16),conv_sub_blocks=(2,2))[16]
# sim middle at 600nm
m = SimLSM_Cylindrical(dn=dn, signal=grid, NA_illum=0.1, NA_detect=0.4, size=(phys_dim,)*3, n0=1.33,
                       lam_detect=0.6, lam_illum=0.6)
image600nm = m.simulate_image_z(cz=0, psf_grid_dim=(16,16),conv_sub_blocks=(2,2))[16]

imsave('/Users/david/Desktop/test_400nm.tif', image400nm)
imsave('/Users/david/Desktop/test_600nm.tif', image600nm)

In [ ]:
# Test show lisghtsheet
m = SimLSM_Cylindrical(dn=np.zeros_like(dn), signal=grid, NA_illum=0.1, NA_detect=0.4, size=(phys_dim,)*3, n0=1.33,
                       zfoc_illum=phys_dim*3/4)

In [ ]:
ill = m.propagate_illum()

In [ ]:
plt.imshow(ill[:,256])